In [ ]:
from pathlib import Path
from utils.plots import plot_confusion_matrix
path = Path("models").joinpath("autosave").joinpath("2022-05-30_15-18-12").joinpath("confmat.npy")
conf_mat = np.load(str(path))
plot_confusion_matrix(conf_mat, title="Thresholding", normalize=True)